In [1]:
!pip install keras tensorflow

In [1]:
import pandas as pd
from matplotlib import pyplot
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

Using TensorFlow backend.


In [6]:
base_data = pd.read_csv('./training_data_real.csv', delimiter=';')#, parse_dates=['dat_calendar_'])
#base_data = base_data.set_index('dat_calendar_')

In [13]:
stuttgart_base = base_data[base_data['store_id'] == 12125]

In [14]:
stuttgart_reduced = stuttgart_base.iloc[:, [3,4,5]]
stuttgart_reduced.head()

,orig_item_nr,SellCount,AvgTemp
dat_calendar_,,,
2019-02-07,2259,7,0.2
2019-02-07,4678,14,0.2
2019-02-07,120185,14,0.2
2019-02-07,4691,168,0.2
2019-02-07,85353,14,0.2


In [102]:
products = stuttgart_base["item_desc"].unique()

In [15]:
stuttgart_ohne_pfand = stuttgart_reduced.query('orig_item_nr not in [4300,6151,6147,3731]')

In [10]:
test = stuttgart_ohne_pfand.groupby('orig_item_nr').sum()
test.loc[test['SellCount'].idxmax()]

SellCount    359646.0
AvgTemp         608.7
Name: 92781, dtype: float64

In [13]:
#base_data[base_data['orig_item_nr']==92781]

In [8]:
from pandas import DataFrame
from pandas import concat
 
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	"""
	Frame a time series as a supervised learning dataset.
	Arguments:
		data: Sequence of observations as a list or NumPy array.
		n_in: Number of lag observations as input (X).
		n_out: Number of observations as output (y).
		dropnan: Boolean whether or not to drop rows with NaN values.
	Returns:
		Pandas DataFrame of series framed for supervised learning.
	"""
	n_vars = 1 if type(data) is list else data.shape[1]
	df = DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg

In [9]:
reframed =series_to_supervised(stuttgart_ohne_pfand)

In [10]:
# split into train and test sets
values = reframed.values
n_train_hours = 365 * 24
train = values[:n_train_hours, :]
test = values[n_train_hours:, :]
# split into input and outputs
train_X, train_y = train[:, :-1], train[:, -1]
test_X, test_y = test[:, :-1], test[:, -1]
# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

(8760, 1, 5) (8760,) (211772, 1, 5) (211772,)


In [ ]:
# design network
model = Sequential()
model.add(LSTM(50, input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(Dense(1))
model.compile(loss='mae', optimizer='adam')
# fit network
history = model.fit(train_X, train_y, epochs=50, batch_size=72, validation_data=(test_X, test_y), verbose=2, shuffle=False)
# plot history
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
pyplot.show()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Train on 8760 samples, validate on 211772 samples
Epoch 1/50


In [16]:
stuttgart_ohne_pfand.head()

,orig_item_nr,SellCount,AvgTemp
dat_calendar_,,,
2019-02-07,2259,7,0.2
2019-02-07,4678,14,0.2
2019-02-07,120185,14,0.2
2019-02-07,4691,168,0.2
2019-02-07,85353,14,0.2


In [17]:
test = stuttgart_ohne_pfand.groupby('orig_item_nr').sum()
test.loc[test['SellCount'].idxmax()]

SellCount    359646.0
AvgTemp         608.7
Name: 92781, dtype: float64

In [18]:
t1 = stuttgart_ohne_pfand[stuttgart_ohne_pfand['orig_item_nr']==92781]

In [20]:
t1.to_csv('./test_azure.csv')

In [18]:
base_data = pd.read_csv('./training_data_real.csv', delimiter=';')
base_data.head()

,orig_item_nr,item_desc,dat_calendar_,SellCount,AvgTemp,Niederschlagsmenge,Wochentag
0,4300,Pfand 0.25 EM,2019-01-21,2177,-4.8,0.0,1
1,2,Natürliches Mineralwasser still,2019-01-21,648,-4.8,0.0,1
2,92781,Weizenbrötchen [B],2019-01-21,440,-4.8,0.0,1
3,78246,Schoko-Donut,2019-01-21,366,-4.8,0.0,1
4,8,Mineralwasser Medium,2019-01-21,320,-4.8,0.0,1


In [19]:
t1 = base_data.query('orig_item_nr not in [4300,6151,6147,3731]')

In [22]:
t2 = t1[t1['orig_item_nr'] == 126945]
t3 = t2.iloc[:,[0,2,3,4,5,6]]
t3.head()

,orig_item_nr,dat_calendar_,SellCount,AvgTemp,Niederschlagsmenge,Wochentag
19,126945,2019-01-21,142,-4.8,0.0,1
2295,126945,2019-01-24,98,-4.0,0.3,4
4583,126945,2019-01-22,143,-3.8,0.0,2
6785,126945,2019-01-23,67,-3.0,0.6,3
8860,126945,2019-01-25,151,-2.8,0.7,5


In [23]:
t3.to_csv('./training_data.csv')

In [17]:
base_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 255303 entries, 0 to 255302
Data columns (total 7 columns):
orig_item_nr          255303 non-null int64
item_desc             255303 non-null object
dat_calendar_         255303 non-null object
SellCount             255303 non-null int64
AvgTemp               255303 non-null object
Niederschlagsmenge    255303 non-null object
Wochentag             255303 non-null int64
dtypes: int64(3), object(4)
memory usage: 13.6+ MB


In [24]:
t3.shape

(107, 6)

In [28]:
t2 = t1[t1['orig_item_nr'] == 5226]
t3 = t2.iloc[:,[0,2,3,4,5,6]]
t3.to_csv('./training_data.csv')

In [25]:
t3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 107 entries, 19 to 252969
Data columns (total 6 columns):
orig_item_nr          107 non-null int64
dat_calendar_         107 non-null object
SellCount             107 non-null int64
AvgTemp               107 non-null float64
Niederschlagsmenge    107 non-null float64
Wochentag             107 non-null int64
dtypes: float64(2), int64(3), object(1)
memory usage: 5.9+ KB


In [26]:
t4 = t3.resample('W').sum('SellCount')

TypeError: Only valid with DatetimeIndex, TimedeltaIndex or PeriodIndex, but got an instance of 'Int64Index'

In [29]:
t3.head()

,orig_item_nr,dat_calendar_,SellCount,AvgTemp,Niederschlagsmenge,Wochentag
1516,5226,2019-01-21,2,-4.8,0.0,1
3838,5226,2019-01-24,2,-4.0,0.3,4
5409,5226,2019-01-22,5,-3.8,0.0,2
8295,5226,2019-01-23,2,-3.0,0.6,3
10737,5226,2019-01-25,2,-2.8,0.7,5
